In [2]:
import requests
import pandas as pd
from newspaper import Article
from tqdm import tqdm  # progress bar for nicer output
from time import sleep 

In [ ]:
# ## code before hand

# url = "https://api.gdeltproject.org/api/v2/doc/doc"
# params = {
#     "query": "united states",
#     "mode": "artlist",
#     "maxrecords": 250,  # slightly higher in case non-US articles are included
#     "format": "json",
#     "sort": "datedesc",
#     "sourcecountry": "US"
# }

# response = requests.get(url, params=params)

# if response.status_code != 200:
#     print("Error:", response.status_code)
#     print(response.text)
# else:
#     data = response.json()
#     articles = data.get("articles", [])
    
#     # Convert to DataFrame
#     df = pd.DataFrame(articles)

#     # Filter only rows where sourcecountry is "United States"
#     df_us = df[df["sourcecountry"] == "United States"].head(100)

#     # Show selected columns
#     print(df_us[["title", "url", "seendate", "sourcecountry"]])

#     # Save to CSV
#     df_us.to_csv("gdelt_us_articles.csv", index=False)
#     print("Saved to gdelt_us_articles.csv ✅")


                                                 title  \
5    6 . 5 Magnitude Earthquake Reported | 720 THE ...   
7    This Unexpected South Florida City Might Be th...   
11   EU - US trade deal expected to confirm duty - ...   
12   SkyWest balks at prospect of paying steep tari...   
19   Harper says Carney team sought his trade advic...   
..                                                 ...   
237  As U . S . plans to exit UNESCO , Ohio earthwo...   
239  A raw deal but the best they could get with Tr...   
240  US Trade Rep Expects No  Enormous Breakthrough...   
247  In brief :  The Walking Dead : Daryl Dixon  re...   
249  Trump moves to lift visa restrictions for Arge...   

                                                   url          seendate  \
5    https://720thevoice.iheart.com/content/2025-07...  20250728T201500Z   
7    https://www.miaminewtimes.com/restaurants/this...  20250728T201500Z   
11   https://article.wn.com/view/2025/07/28/EUUS_tr...  20250728T201500Z   

In [3]:
# ## appears to be a test for opening the url of articles
    
# url = "https://www.sbsun.com/2025/07/28/trump-gaza-food/"
# article = Article(url)
# article.download()
# article.parse()

# print(article.title)
# print(article.text)

In [ ]:
# # Step 1: Query GDELT Doc API for recent US articles
# url = "https://api.gdeltproject.org/api/v2/doc/doc"
# params = {
#     "query": "trump AND harris AND sourcelang:english AND theme:ELECTION AND sourcecountry:US",
#     "mode": "artlist",
#     "maxrecords": 250,
#     "format": "json",
#     "sort": "datedesc",
#     "STARTDATETIME" : 20241103000000,
#     "ENDDATETIME" : 20241104000000
# }

# response = requests.get(url, params=params)
# if response.status_code != 200:
#     print("Error:", response.status_code)
#     print(response.text)
#     exit()

# data = response.json()
# articles = data.get("articles", [])

# # Step 2: Convert to DataFrame
# df = pd.DataFrame(articles)

# # Filter only "United States" sourcecountry and limit to 100 articles
# df_us = df#[df["sourcecountry"] == "United States"]#.head(100)

# # # Step 3: Extract article content using newspaper3k
# # def extract_content(article_url):
# #     try:
# #         article = Article(article_url)
# #         article.download()
# #         article.parse()
# #         return article.text
# #     except Exception as e:
# #         return None  # or you can return str(e) to debug failures

# # print("Extracting article content for each URL... This may take some time.")
# # df_us["content"] = [extract_content(url) for url in tqdm(df_us["url"])]

# # # Optional: Drop articles with no content
# # df_us = df_us.dropna(subset=["content"])


# # Step 4: Save to CSV
# df_us.to_csv("C:\\Users\\Woochoel Shin\\Pictures\\Ezras stuff temporary\\coding\\Stemforall\\gdelt_us_articles_sorted.csv", index=True)#, quotechar=""")
# print("Saved articles with content to gdelt_us_articles_ezra.csv")





Extracting article content for each URL... This may take some time.


100%|██████████| 250/250 [04:01<00:00,  1.03it/s]

Saved articles with content to gdelt_us_articles_ezra.csv


In [ ]:
### scraper for a year
import requests
import pandas as pd
from time import sleep 
import os

###
# ANNOUNCEMENT: when scraping for other elections, make sure to change the year and the querry to the candidates for that election cycle
###


# Query GDELT Doc API for recent US articles
url = "https://api.gdeltproject.org/api/v2/doc/doc"
days_in_month = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 4] # number of days in each month to make the nested for loops easier to implement
# november only has ten days because we don't need to get articles from past election day

year = 2012 ### CHANGE THIS DEPENDING ON THE ELECTION CYCLE
candidates = "obama AND romney" ### CHANGE THIS DEPENDING ON THE ELECTION CYCLE

# creating folders for the year if it already doesn't exist
path = f"us_articles\\{year} election cycle"
try:
    os.makedirs(path)
    print(f"Folders '{path}' created successfully.")
except FileExistsError:
    print(f"Folders '{path}' already exist.")
except OSError as e:
    print(f"Error creating folders: {e}")

### loop through jan-nov
for month in range(4, 12): # exclude december bc the election results are out by then
    
    # creating folders for each month if it doesnt exist already
    path = f"us_articles\\{year} election cycle\\month {month}"
    try:
        os.makedirs(path)
        print(f"Folders '{path}' created successfully.")
    except FileExistsError:
        print(f"Folders '{path}' already exist.")
    except OSError as e:
        print(f"Error creating folders: {e}")

    ### each day gets its own dataset
    for day in range(1, days_in_month[month-1]+1): #plus one error on the months bc the list indexes starting with 0, but january is represented by 1 and plus one error on the number of days in the month because range() is non inclusive at the end
        
        # code to reset the DFs and adding a header only
        response = requests.get(url, params={"query":"icecream","mode":"artlist","maxrecords": 1,"format": "json"})
        
        sleeptime = 45 # sometimes the api softlocks us from making requests, so we have to wait it out
        while response.status_code == 429:
            print(f"sleeping for {sleeptime} seconds, wait 5 seconds my ass")
            sleep(sleeptime)
            response = requests.get(url, params={"query":"icecream","mode":"artlist","maxrecords": 1,"format": "json"})
            sleeptime += 5

        data = response.json()
        article = data.get("articles", [])
        me = pd.DataFrame(article)
        me=me.drop(0)
        me.to_csv(f"C:\\Users\\Woochoel Shin\\Pictures\\Ezras stuff temporary\\coding\\Stemforall\\us_articles\\{year} election cycle\\month {month}\\{year}-{month}-{day}_gdelt_articles.csv", index=False)#, quotechar=""")
        
        # changes start and end to be modified by fstrings, not math. 
        # this means you can now start the scraping process at any date by adjusting the range of the nested for loops
        start = int(f"{year}{month:02d}{day:02d}000000")
        end = int(f"{year}{month:02d}{day:02d}003000")

        ### scrape every hour within a day, then add it to the "day" csv file
        for hour in range(0,24): #idr why this starts at 0 while the other two loops start at 1
            
            # reaching out to the api
            params = {
                "query": f"{candidates} AND sourcelang:english AND theme:ELECTION AND sourcecountry:US",
                "mode": "artlist",
                "maxrecords": 250,
                "format": "json",
                "sort": "datedesc",
                "STARTDATETIME" : start, 
                "ENDDATETIME" : end #for some reason, gdelt gives us articles 45 minutes past the endtime i give it
            }
            
            response = requests.get(url, params=params)

            # waiting out the softlock, previously thought to be 45 sec, but when we increased the wait time to 45, we still ended up waiting for a total of 520 seconds. 
            # new hypothesis is now 500 seconds
            sleeptime = 500
            while response.status_code == 429:
                print(f"sleeping for {sleeptime} seconds, wait 5 seconds my ass")
                sleep(sleeptime)
                response = requests.get(url, params=params)
                sleeptime += 5

            if response.status_code != 200:
                print("Error:", response.status_code)
                print(response.text)
                exit()
                
            # print("Error:", response.status_code)
            # print(response.text)

            data = response.json()
            articles = data.get("articles", [])

            # Convert to DataFrame
            df = pd.DataFrame(articles)

            # append this hour's worth of articles to the day csv
            df.to_csv(f"C:\\Users\\Woochoel Shin\\Pictures\\Ezras stuff temporary\\coding\\Stemforall\\us_articles\\{year} election cycle\\month {month}\\{year}-{month}-{day}_gdelt_articles.csv", mode='a', header=False, index=False)#, quotechar=""")
            
            # print the number of articles that we downloaded from that hour
            # print(f"{hour}. Saved {str(len(df))} articles from time {start} to {end}")

            # move up by an hour. the time resets when start,end is set
            start += 10000
            end += 10000

        # stupid method to index the entire thing. reads the entire csv, then it replaces that same csv but now with indexes
        df_day = pd.read_csv(f"C:\\Users\\Woochoel Shin\\Pictures\\Ezras stuff temporary\\coding\\Stemforall\\us_articles\\{year} election cycle\\month {month}\\{year}-{month}-{day}_gdelt_articles.csv")
        df_day.to_csv(f"C:\\Users\\Woochoel Shin\\Pictures\\Ezras stuff temporary\\coding\\Stemforall\\us_articles\\{year} election cycle\\month {month}\\{year}-{month}-{day}_gdelt_articles.csv", header=True, index=True)

        # prints number of articles downloaded on each day
        print(f"day {day} done: {len(df_day)} articles downloaded")

    # keep track of what months are done
    print(f"month {month} done")

Folders 'us_articles\2012 election cycle' already exist.
Folders 'us_articles\2012 election cycle\month 4' already exist.
Error: 200
Invalid query start date.



JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [3]:
### adding sentiment analysis
import requests
import pandas as pd
from newspaper import Article
from tqdm import tqdm  # progress bar for nicer output
from time import sleep 
from textblob import TextBlob


days_in_month = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 4] # number of days in each month to make the nested for loops easier to implement

file_path = "C:\\Users\\Woochoel Shin\\Pictures\\Ezras stuff temporary\\coding\\Stemforall"
year = 2024

def extract_content(article_url):
    try:
        article = Article(article_url)
        article.download()
        article.parse()
        return article.text
    except Exception as e:
        return None  # or you can return str(e) to debug failures

for month in range(1, 2): # exclude december bc the election results are out by then

    ### each day gets its own dataset
    for day in range(1, 2):#days_in_month[month-1]+1): #plus one error on the months bc the list indexes starting with 0, but january is represented by 1 and plus one error on the number of days in the month because range() is non inclusive at the end
        
        df_day = pd.read_csv('z_test_gdelt_us_articles.csv')#f"{file_path}\\us_articles\\{year} election cycle\\month {month}\\{year}-{month}-{day}_gdelt_articles.csv")

        # df_day['content'] = Article(df_day['url'].str)

        print("Extracting article content for each URL... This may take some time.")
        df_day["content"] = [extract_content(url) for url in tqdm(df_day["url"])]

        # Optional: Drop articles with no content
        # df_day = df_day.dropna(subset=["content"])

df_day.to_csv("testing.csv")

Extracting article content for each URL... This may take some time.


  0%|          | 0/14 [00:00<?, ?it/s]

100%|██████████| 14/14 [00:12<00:00,  1.15it/s]


In [2]:
### this is the code for scraping a single day
import requests
import pandas as pd
from time import sleep 

url = "https://api.gdeltproject.org/api/v2/doc/doc"

# code to reset the DF
response = requests.get(url, params={"query": "icecream","mode": "artlist","maxrecords": 1,"format": "json"})
data = response.json()
article = data.get("articles", [])
me = pd.DataFrame(article)
me=me.drop(0)
me.to_csv("C:\\Users\\Woochoel Shin\\Pictures\\Ezras stuff temporary\\coding\\Stemforall\\z_test_gdelt_us_articles.csv", index=False)

# me = pd.DataFrame()
# me.to_csv("C:\\Users\\Woochoel Shin\\Pictures\\Ezras stuff temporary\\coding\\Stemforall\\test_gdelt_us_articles.csv", index=False)

start = 20240101000000 #20241103000000
end = 20240101003000 #20241103003000
count = 0

for hour in range(0,24):
    params = {
        "query": "trump AND harris AND sourcelang:english AND theme:ELECTION AND sourcecountry:US",
        "mode": "artlist",
        "maxrecords": 250,
        "format": "json",
        "sort": "datedesc",
        "STARTDATETIME" : start, 
        "ENDDATETIME" : end #for some reason, gdelt gives us articles 45 minutes past the endtime i give it
    }

    response = requests.get(url, params=params)
    if response.status_code != 200:
        print("Error:", response.status_code)
        print(response.text)
        exit()

    # print("Error:", response.status_code)
    # print(response.text)

    data = response.json()
    articles = data.get("articles", [])

    # Step 2: Convert to DataFrame
    df = pd.DataFrame(articles)

    # print(df)

    # Step 4: Save to CSV
    df.to_csv(f"C:\\Users\\Woochoel Shin\\Pictures\\Ezras stuff temporary\\coding\\Stemforall\\z_test_gdelt_us_articles.csv", mode='a', header=False, index=False)#, quotechar=""")
    
    # print the number of articles that we downloaded from that hour
    print(f"hour {count}. Saved {str(len(df))} articles from time {start} to {end}")


    start += 10000
    end += 10000
    count+=1

# stupid method to index the entire thing
bruh = pd.read_csv("C:\\Users\\Woochoel Shin\\Pictures\\Ezras stuff temporary\\coding\\Stemforall\\z_test_gdelt_us_articles.csv")
bruh.to_csv("C:\\Users\\Woochoel Shin\\Pictures\\Ezras stuff temporary\\coding\\Stemforall\\z_test_gdelt_us_articles.csv", header=True, index=True)
print(len(bruh))

Error: 200
{}
hour 0. Saved 0 articles from time 20240101000000 to 20240101003000
Error: 200
{}
hour 1. Saved 0 articles from time 20240101010000 to 20240101013000
Error: 200
{}
hour 2. Saved 0 articles from time 20240101020000 to 20240101023000
Error: 200
{"articles": [ { "url": "https://www.fitsnews.com/2023/12/31/palmetto-political-stock-index-previewing-a-pivotal-election-year/", "url_mobile": "", "title": "Palmetto Political Stock Index : Previewing A Pivotal Election Year", "seendate": "20240101T030000Z", "socialimage": "https://www.fitsnews.com/wp-content/uploads/2023/12/gettyimages-1860116039-1024x768.jpg", "domain": "fitsnews.com", "language": "English", "sourcecountry": "United States" }] }
hour 3. Saved 1 articles from time 20240101030000 to 20240101033000
Error: 200
{}
hour 4. Saved 0 articles from time 20240101040000 to 20240101043000
Error: 200
{}
hour 5. Saved 0 articles from time 20240101050000 to 20240101053000
Error: 200
{"articles": [ { "url": "https://legalinsurrect

In [ ]:
# how many of those articles actually exist
url = "https://api.gdeltproject.org/api/v2/doc/doc"
params = {
    "query": "trump AND harris",
    "mode": "TimelineVolRaw",
    "maxrecords": 250,
    "format": "json",
    "sort": "datedesc",
    "sourcecountry": "United States",
    "sourcelang" : "english",
    "theme" : "ELECTION",
    "STARTDATETIME" : 20240904000000,
    "ENDDATETIME" : 20240904010000#20241104000000
}

response = requests.get(url, params=params)
print(response.type())